# 管道里使用的
## RunnableLambda：把方法转成Runnable方法,对数据的处理
## RunnableParallel：生成Runnable字典，可以添加额外参数
## RunnablePassthrough：传递参数
### RunnablePassthrough.assign，传递参数输出的同时，可以添加额外参数

In [2]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

# RunnableParallel,生成Runnable字典（可以添加字段）
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x + 1
)

runnable.invoke(1)  # {'origin': 1, 'modified': 2}

{'origin': 1, 'modified': 2}

In [3]:
def fake_llm(prompt: str) -> str:  # Fake LLM for the example
    return "123456"


# 'parsed': lambda text: text[::-1] 它定义了一个匿名函数（lambda函数），用于字符串的反转操作。
# [::-1]意味着从开始到结束，步长为-1，即反向遍历字符串，从而实现字符串的反转。
# text的值其实就是original的值
chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(),  # Original LLM output
    'parsed': lambda text: text[::-1]  # Parsing logic
}

chain.invoke('hello')  # {'original': 'completion', 'parsed': 'noitelpmoc'}

{'original': '123456', 'parsed': '654321'}

In [4]:
from langchain_core.runnables import RunnablePassthrough


def fake_llm(prompt: str) -> str:  # Fake LLM for the example
    return "123456"


# RunnablePassthrough.assign: 在某些情况下，在传递输入的同时向输出添加一些键可能会很有用。在这种情况下，您可以使用“assign”方法：
runnable = {
               'llm1': fake_llm,
               'llm2': fake_llm,
           } | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

{'llm1': '123456', 'llm2': '123456', 'total_chars': 12}

In [5]:
runnable = {
               'llm1': fake_llm,
               'llm2': fake_llm,
           } | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
)

runnable.invoke('hello')

{'llm1': '123456', 'llm2': '123456', 'total_chars': 12}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

model = ChatOpenAI()
joke_chain = (
        ChatPromptTemplate.from_template("tell me a joke about {topic}")
        | model
)
poem_chain = (
        ChatPromptTemplate.from_template("write a 2-line poem about {topic}")
        | model
)

runnable = RunnableParallel(joke=joke_chain, poem=poem_chain)

# {key: "" for ...}: 这是字典推导式的结构。对于runnable.output_schema()中的每一个键，它都会创建一个新的键值对，
# 其中键是迭代中得到的键，值则是空字符串""。
output = {key: "" for key, _ in runnable.output_schema()}
for chunk in runnable.stream({"topic": "bear"}):
    for key in chunk:
        output[key] = output[key] + chunk[key].content
    print(output)  # noqa: T201

In [6]:
# This is a RunnableAssign
from typing import Dict
from langchain_core.runnables.passthrough import (
    RunnableAssign,
    RunnableParallel,
)
from langchain_core.runnables.base import RunnableLambda


def add_ten(x: Dict[str, int]) -> Dict[str, int]:
    return {"added": x["input"] + 10}


mapper = RunnableParallel(
    {"add_step": RunnableLambda(add_ten), }
)

runnable_assign = RunnableAssign(mapper)

# Synchronous example
runnable_assign.invoke({"input": 5})
# returns {'input': 5, 'add_step': {'added': 15}}

# Asynchronous example
# await runnable_assign.ainvoke({"input": 5})
# returns {'input': 5, 'add_step': {'added': 15}}

{'input': 5, 'add_step': {'added': 15}}

In [ ]:
# _inputs = RunnableParallel(
#   standalone_question=RunnablePassthrough.assign(
#     chat_history=lambda x: _format_chat_history(x["chat_history"])
#   )
#                       | CONDENSE_QUESTION_PROMPT
#                       | llm
#                       | StrOutputParser(),
# )
# 
# _context = {
#   "context": itemgetter("standalone_question") | retriever | _combine_documents,
#   "question": lambda x: x["standalone_question"],
# }
# 
# chain = _inputs | _context | LLM_CONTEXT_PROMPT | llm | StrOutputParser()
# 
# chain = chain.with_types(input_type=ChainInput)